In [113]:
import json
import requests
import pandas as pd
import numpy as np

In [98]:
BASE_URL = 'https://sofascore.com/api/v1/event'

stats = requests.get((BASE_URL + '/11352406/statistics')).json()
event = requests.get((BASE_URL + '/11352406')).json()

In [99]:
df = pd.json_normalize(stats.get('statistics')[0].get('groups'), record_path=['statisticsItems']) # ALL period
df.set_index("name", inplace=True)
df = df.T.iloc[:2, :].reset_index()

In [102]:
cool = pd.json_normalize(event)

In [110]:
home = cool['event.homeTeam.name']
away = cool['event.awayTeam.name']

In [111]:
df

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Aerial duels,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks
0,home,74%,4.32,5,37,1,12,7,763,20,...,9/23 (39%),10/18 (56%),65%,20,5,52,3,1,0.23,1
1,away,26%,0.51,2,4,9,1,6,276,13,...,14/23 (61%),7/11 (64%),54%,13,8,44,28,9,-1.18,16


In [114]:
df['team'] = np.where(df['index'] == 'home', home, away)

In [115]:
df

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks,team
0,home,74%,4.32,5,37,1,12,7,763,20,...,10/18 (56%),65%,20,5,52,3,1,0.23,1,Manchester City
1,away,26%,0.51,2,4,9,1,6,276,13,...,7/11 (64%),54%,13,8,44,28,9,-1.18,16,Luton Town


In [101]:
list(pd.json_normalize(event).columns)

['event.tournament.name',
 'event.tournament.slug',
 'event.tournament.category.name',
 'event.tournament.category.slug',
 'event.tournament.category.sport.name',
 'event.tournament.category.sport.slug',
 'event.tournament.category.sport.id',
 'event.tournament.category.id',
 'event.tournament.category.country.alpha2',
 'event.tournament.category.country.alpha3',
 'event.tournament.category.country.name',
 'event.tournament.category.flag',
 'event.tournament.category.alpha2',
 'event.tournament.uniqueTournament.name',
 'event.tournament.uniqueTournament.slug',
 'event.tournament.uniqueTournament.primaryColorHex',
 'event.tournament.uniqueTournament.secondaryColorHex',
 'event.tournament.uniqueTournament.category.name',
 'event.tournament.uniqueTournament.category.slug',
 'event.tournament.uniqueTournament.category.sport.name',
 'event.tournament.uniqueTournament.category.sport.slug',
 'event.tournament.uniqueTournament.category.sport.id',
 'event.tournament.uniqueTournament.category.id

In [53]:
stats = pd.json_normalize(df.explode('statisticsItems')['statisticsItems'])

In [80]:
stats.head()

,name,home,away,compareCode,statisticsType,valueType,homeValue,awayValue,renderType,key,homeTotal,awayTotal
0,Ball possession,74%,26%,1,positive,event,74.00,26.00,2,ballPossession,NaN,NaN
1,Expected goals,4.32,0.51,1,positive,event,4.32,0.51,1,expectedGoals,NaN,NaN
2,Big chances,5,2,1,positive,event,5.00,2.00,1,bigChanceCreated,NaN,NaN
3,Total shots,37,4,1,positive,event,37.00,4.00,1,totalShotsOnGoal,NaN,NaN
4,Goalkeeper saves,1,9,2,positive,event,1.00,9.00,1,goalkeeperSaves,NaN,NaN


In [85]:
stats.set_index("name", inplace=True)



KeyError: "None of ['name'] are in the columns"

In [90]:
# Transpose the DataFrame
stats.T.iloc[:2, :].reset_index()

name,index,Ball possession,Expected goals,Big chances,Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,...,Aerial duels,Dribbles,Tackles won,Total tackles,Interceptions,Recoveries,Clearances,Total saves,Goals prevented,Goal kicks
0,home,74%,4.32,5,37,1,12,7,763,20,...,9/23 (39%),10/18 (56%),65%,20,5,52,3,1,0.23,1
1,away,26%,0.51,2,4,9,1,6,276,13,...,14/23 (61%),7/11 (64%),54%,13,8,44,28,9,-1.18,16


In [24]:
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [64]:
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.set_page_load_timeout(10)

try:
    driver.get("https://www.sofascore.com/player/erling-haaland/839956")
except:
    pass


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [65]:
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [66]:
for x in logs:
    path = x['params'].get('headers', {}).get(':path', '') # Extract the ':path' value from the headers, defaulting to an empty string if not found
    if '/api/' in path:
        print(path)

/api/v1/team/17/image
/api/v1/player/839956/statistics/seasons
/api/v1/country/alpha2
/api/v1/sport/-14400/event-count
/api/v1/player/839956/attribute-overviews
/api/v1/player/839956/events/last/0
/api/v1/player/839956/events/next/0
/api/v1/player/839956/last-year-summary
/api/v1/player/839956/national-team-statistics
/api/v1/player/839956/statistics/seasons
/api/v1/player/839956/image
/api/v1/team/0/image
/api/v1/team/663/image
/api/v1/team/661/image
/api/v1/team/2046/image
/api/v1/team/2673/image
/api/v1/player/839956/media
/api/201540/envelope/?sentry_key=d693747a6bb242d9bb9cf7069fb57988&sentry_version=7&sentry_client=sentry.javascript.nextjs%2F7.113.0
/api/v1/unique-tournament/17/image/dark
/api/v1/unique-tournament/7/image/dark
/api/v1/unique-tournament/27/image/dark
/api/v1/unique-tournament/19/image/dark
/api/v1/unique-tournament/851/image/dark
/api/v1/unique-tournament/853/image/dark
/api/v1/unique-tournament/346/image/dark
/api/v1/unique-tournament/465/image/dark
/api/v1/team/

In [78]:
import requests

response = requests.get('https://www.sofascore.com/api/v1/player/839956/penalty-history/unique-tournament/17/season/52186').json()



In [79]:
response

{'penalties': [{'event': {'tournament': {'name': 'Premier League',
     'slug': 'premier-league',
     'category': {'name': 'England',
      'slug': 'england',
      'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
      'id': 1,
      'flag': 'england',
      'alpha2': 'EN'},
     'uniqueTournament': {'name': 'Premier League',
      'slug': 'premier-league',
      'primaryColorHex': '#3c1c5a',
      'secondaryColorHex': '#f80158',
      'category': {'name': 'England',
       'slug': 'england',
       'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
       'id': 1,
       'flag': 'england',
       'alpha2': 'EN'},
      'userCount': 1699210,
      'id': 17,
      'displayInverseHomeAwayTeams': False},
     'priority': 600,
     'isLive': False,
     'id': 1},
    'customId': 'lI',
    'status': {'code': 100, 'description': 'Ended', 'type': 'finished'},
    'winnerCode': 2,
    'homeTeam': {'name': 'Tottenham Hotspur',
     'slug': 'tottenham-hotspur',
     'shortN